Hello，同学们，通过第一节课的学习，你一定学到了不少实操的技能吧！那么我们趁热打铁，赶紧把本节课学到的技能应用起来吧！

本节课的作业要求：

+ 参考课上的实战代码实现对教育栏目下的新闻正文内容的爬取，并以文件的形式保存，并且选择任何一种分词方式对每篇新闻进行分词，借助Sklearn中的工具实现每篇新闻的TF-IDF的向量表示。
+ 认真阅读注释，对你做对该题至关重要
+ 作业已经给出大部分程序实现代码，你只需要在`######## your code ~n line ########` 与 `######## your code end ########` 行之间根据提示补充完毕相应的代码即可。

# 1 语料获取途径-爬虫

### 使用 requests 请求库对人民网教育栏目下[滚动新闻](http://edu.people.com.cn/GB/227065/index1.html)的爬取

* 第一步：安装requests请求库

In [ ]:
! pip install requests==2.28.1

+ 第二步：编写请求函数

In [ ]:
import requests
import random
# 模拟浏览器请求头，这里提供了一定量的请求头存储在.data/user_agent.txt中，来模仿不同的浏览器发送请求，应对反爬虫机制
user_agents = [ua.strip() for ua in open('data/user_agent.txt', 'r', encoding='utf-8').readlines()]
def request_server(url, user_agents=user_agents):
    """
    模拟浏览器发送get请求
    :param url: 请求的链接
    :return: 服务器返回的数据封装城的Response对象
    """
    headers = {'user-agent':random.choice(user_agents)}
    response = requests.get(url=url, headers=headers)
    return response
    

+ 第三步：构造解析详情页链接的函数

In [ ]:
# 这里用lxml演示,首先安装lxml
! pip install lxml==4.9.1

In [1]:
# from lxml import etree
from lxml import html
def extract_detail_url(html_str):
    """
    解析详每页中的详情页链接
    :param html_str: 每页的网页源代码
    :return: 当前页详情页链接
    """
    tree = html.fromstring(html_str)  # /html/body/div[4]/div[1]/div[2]/ul/li/a/@href
    d_urls = tree.xpath('/html/body/div[5]/div[1]/div[2]/ul/li/a/@href')
    for d_url in d_urls:
        yield d_url

+ 第四步：构造解析文章内容的函数

### 问题一：填写匹配正文内容的xpath路径。

In [ ]:
def extract_content(html_str):
    """
    提取正文内容
    :param html_str:网页源代码
    :return:正文内容
    """
    tree = html.fromstring(html_str)  # /html/body/div[1]/div[7]/div[1]/div[3]/p/text()
    ######## your code ~ 1 line  ######## 提示：这里填写匹配文章正文内容的xpath路径
    documents = 
    ######## your code end ########
    return documents

+ 第五步：编写主函数
    + 整个设计思路是while层构造每页的链接，for层构造每页中详情页的链接

### 问题二：去除文本中的特殊字符。

In [ ]:
def main():
#     base_url = 'http://military.people.com.cn'  # http://edu.people.com.cn/n1/2022/0726/c1006-32485754.html
    base_url = 'http://edu.people.com.cn'
    page_index = 1
    data_path = 'data/news_edu.txt'
    f = open(data_path, 'w', encoding='utf-8')
    while True:
        # 构造每页url
        url = base_url + '/GB/227065/index%s.html' % page_index
        response = request_server(url=url)
        # 判断异常页面（超出正常页码范围会返回404），跳出翻页
        if response.status_code != 200: 
            break
        for d_url in extract_detail_url(response.text):
            try:
                # 构造详情页链接
                d_url = base_url + d_url
                response = request_server(url=d_url)
                # 编码设置，不然会乱吗
                response.encoding = 'gbk'
                cs = extract_content(response.text)
                 ######## your code ~1 line ######## 提示：这里通过列表推导式的方式，去除cs列表中每一个元素中的\n\t
                cs = 
                 ######## your code end ########
                f.write(''.join(cs) + '\n')
            except Exception as e:
                print(e.__str__())
        print('第%s页完成。' % page_index)
        page_index += 1
    f.close()

In [ ]:
main()

# 2 中文分词

In [ ]:
# 安装jieba分词工具
!pip install jieba==0.42.1

### 问题三：实现jieba的三种分词模式。
+ jieba分词使用演示。https://github.com/fxsjy/jieba

In [ ]:
import jieba
# 实现分词函数
def tokenizer(text, mode=1):
    # text：type is str
    # return tokens :type is list
    if mode==1:
        #精确模式
        ######## your code ~1 line ######## 提示：实现jieba分词的精确模式
        tokens = 
        ######## your code end ########
    elif mode==2:
        #全模式
        ######## your code ~1 line ######## 提示：实现jieba分词的全模式
        tokens = 
        ######## your code end ########
    else:
        #搜索引擎模式
        ######## your code ~1 line ######## 提示：实现jieba分词的搜索引擎模式
        tokens = 
        ######## your code end ########
    return tokens

In [ ]:
def load_copus(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

In [ ]:
# 加载语料
data_path = 'data/news_edu.txt'
documents = load_copus(data_path)
# 对语料进行分词
documents = [' '.join(tokenizer(doc)) for doc in documents]
documents[4]

# 3 TF-IDF实现

In [ ]:
!pip install scikit-learn

### 问题四：使用Scikit-learn完成爬取文本documents的TF-IDF表示。
+ 可以参考[TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)的官方文档，完成本次作业。

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
# 利用fit_transform得到TF-IDF矩阵
######## your code ~1 line ########
tfidf_matrix = 
######## your code end ######## 

# 利用get_feature_names得到词表
######## your code ~1 line ########

######## your code end ########
# 得到每个单词所对应的ID
# print(tfidf_vec.vocabulary_)
 
# 输出TF-IDF矩阵
# print(tfidf_matrix)